This dataset is from kaggle. To run my code, first you must follow these steps: https://saturncloud.io/blog/how-to-import-kaggle-datasets-into-jupyter-notebook/

## My Data Plan

- Inspect data
- Clean data
- Add variables of interest 
- Visualize the three indicators of interest 
- Identify the accounts with anomalies
- Estimate fraud loss
- Conduct a cost-benefit analysis
- Recommend a decision threshold

In [1]:
pip install kaggle

     |████████████████████████████████| 181 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 7.0 MB/s  eta 0:00:01
     |████████████████████████████████| 418 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 9.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
!kaggle datasets download -d valakhorasani/bank-transaction-dataset-for-fraud-detection

Dataset URL: https://www.kaggle.com/datasets/valakhorasani/bank-transaction-dataset-for-fraud-detection
License(s): apache-2.0
  0%|                                                | 0.00/102k [00:00<?, ?B/s]
100%|█████████████████████████████████████████| 102k/102k [00:00<00:00, 135MB/s]


In [6]:
!unzip bank-transaction-dataset-for-fraud-detection.zip

Archive:  bank-transaction-dataset-for-fraud-detection.zip
  inflating: bank_transactions_data_2.csv  


In [7]:
import pandas as pd
data = pd.read_csv('bank_transactions_data_2.csv')
print(data.head())

  TransactionID AccountID  TransactionAmount      TransactionDate  \
0      TX000001   AC00128              14.09  2023-04-11 16:29:14   
1      TX000002   AC00455             376.24  2023-06-27 16:44:19   
2      TX000003   AC00019             126.29  2023-07-10 18:16:08   
3      TX000004   AC00070             184.50  2023-05-05 16:32:11   
4      TX000005   AC00411              13.45  2023-10-16 17:51:24   

  TransactionType   Location DeviceID      IP Address MerchantID Channel  \
0           Debit  San Diego  D000380  162.198.218.92       M015     ATM   
1           Debit    Houston  D000051     13.149.61.4       M052     ATM   
2           Debit       Mesa  D000235  215.97.143.157       M009  Online   
3           Debit    Raleigh  D000187  200.13.225.150       M002  Online   
4          Credit    Atlanta  D000308    65.164.3.100       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0           70             Doctor                   81 

## Basic Data Cleaning Steps

- Check for missing values
- Check for duplicates
- Check for data consistency/logic
- Check data types
- Clean any categorical fields (e.g. trim spaces)
- Check distributions and outliers

In [23]:
# Check date time objects
pd.to_datetime(data['TransactionDate'], errors='coerce').isna().sum()
pd.to_datetime(data['PreviousTransactionDate'], errors='coerce').isna().sum()

0

In [24]:
# Check missing values
print(data.isnull().sum())

TransactionID              0
AccountID                  0
TransactionAmount          0
TransactionDate            0
TransactionType            0
Location                   0
DeviceID                   0
IP Address                 0
MerchantID                 0
Channel                    0
CustomerAge                0
CustomerOccupation         0
TransactionDuration        0
LoginAttempts              0
AccountBalance             0
PreviousTransactionDate    0
dtype: int64


In [25]:
# Check duplicates
print("Duplicate rows:", data.duplicated().sum())

Duplicate rows: 0


In [26]:
# Check logic of transaction dates 
print("Invalid date order:", (data['TransactionDate'] < data['PreviousTransactionDate']).sum())

Invalid date order: 2512


Looks like we can't rely on the column Previous Transaction Date - all the entries are the same, probably due to an error so this column isn't really meaningful to us anymore.

In [27]:
data_cleaned = data.drop('PreviousTransactionDate', axis=1)

In [28]:
# Check invalid transaction amounts (negatives or zeroes)
print("Invalid transaction amounts:", (data_cleaned['TransactionAmount'] <= 0).sum())

Invalid transaction amounts: 0


I think the data is relatively clean now so I'm going to now add in some variables I'm interested in. 
But before I do that, I'm going to explore the data a bit because I'm not too familiar with fraud analytics.

In [29]:
# I lost a lot of data with the missing PreviousTransactionDate data so now I'm curious if there's multiple transaction rows per account in this dataset.
transactions_per_account = data_cleaned.groupby('AccountID').size()
print(transactions_per_account)

AccountID
AC00001    2
AC00002    7
AC00003    5
AC00004    9
AC00005    9
          ..
AC00496    3
AC00497    6
AC00498    8
AC00499    7
AC00500    4
Length: 495, dtype: int64


# Adding in Some Variables of Interest

In [41]:
# Flagging high value transactions
threshold = data_cleaned['TransactionAmount'].quantile(0.95)
data_cleaned['IsHighValueTransaction'] = (data_cleaned['TransactionAmount'] > threshold).astype(int)
print(data_cleaned.head())

  TransactionID AccountID  TransactionAmount      TransactionDate  \
0      TX000001   AC00128              14.09  2023-04-11 16:29:14   
1      TX000002   AC00455             376.24  2023-06-27 16:44:19   
2      TX000003   AC00019             126.29  2023-07-10 18:16:08   
3      TX000004   AC00070             184.50  2023-05-05 16:32:11   
4      TX000005   AC00411              13.45  2023-10-16 17:51:24   

  TransactionType   Location DeviceID      IP Address MerchantID Channel  \
0           Debit  San Diego  D000380  162.198.218.92       M015     ATM   
1           Debit    Houston  D000051     13.149.61.4       M052     ATM   
2           Debit       Mesa  D000235  215.97.143.157       M009  Online   
3           Debit    Raleigh  D000187  200.13.225.150       M002  Online   
4          Credit    Atlanta  D000308    65.164.3.100       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0           70             Doctor                   81 

In [42]:
# Var to see how much the transaction is compared to the balance
data_cleaned['TransAmountToBalanceRatio'] = data_cleaned['TransactionAmount'] / data_cleaned['AccountBalance']
print(data_cleaned.head())

  TransactionID AccountID  TransactionAmount      TransactionDate  \
0      TX000001   AC00128              14.09  2023-04-11 16:29:14   
1      TX000002   AC00455             376.24  2023-06-27 16:44:19   
2      TX000003   AC00019             126.29  2023-07-10 18:16:08   
3      TX000004   AC00070             184.50  2023-05-05 16:32:11   
4      TX000005   AC00411              13.45  2023-10-16 17:51:24   

  TransactionType   Location DeviceID      IP Address MerchantID Channel  \
0           Debit  San Diego  D000380  162.198.218.92       M015     ATM   
1           Debit    Houston  D000051     13.149.61.4       M052     ATM   
2           Debit       Mesa  D000235  215.97.143.157       M009  Online   
3           Debit    Raleigh  D000187  200.13.225.150       M002  Online   
4          Credit    Atlanta  D000308    65.164.3.100       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0           70             Doctor                   81 

In [45]:
# Flagging balance ratios that are too close to 0
threshold = data_cleaned['TransAmountToBalanceRatio'].quantile(0.95)
data_cleaned['IsHighValueTransaction_2'] = (data_cleaned['TransAmountToBalanceRatio'] > threshold).astype(int)
print(data_cleaned.head())

  TransactionID AccountID  TransactionAmount      TransactionDate  \
0      TX000001   AC00128              14.09  2023-04-11 16:29:14   
1      TX000002   AC00455             376.24  2023-06-27 16:44:19   
2      TX000003   AC00019             126.29  2023-07-10 18:16:08   
3      TX000004   AC00070             184.50  2023-05-05 16:32:11   
4      TX000005   AC00411              13.45  2023-10-16 17:51:24   

  TransactionType   Location DeviceID      IP Address MerchantID Channel  \
0           Debit  San Diego  D000380  162.198.218.92       M015     ATM   
1           Debit    Houston  D000051     13.149.61.4       M052     ATM   
2           Debit       Mesa  D000235  215.97.143.157       M009  Online   
3           Debit    Raleigh  D000187  200.13.225.150       M002  Online   
4          Credit    Atlanta  D000308    65.164.3.100       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0           70             Doctor                   81 

In [46]:
data_cleaned.to_csv('data_cleaned.csv', index=False)